<a href="https://colab.research.google.com/github/Nhrkr/Predicting-Social-Unrest/blob/master/Arjunil_copy_last_min_shenanigans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

def create_data_set(_data_set, _look_back=1):
    data_x, data_y = [], []
    for i in range(len(_data_set) - _look_back - 1):
        a = _data_set[i:(i + _look_back), 0]
        data_x.append(a)
        data_y.append(_data_set[i + _look_back, 0])
    return np.array(data_x), np.array(data_y)




In [0]:
data= pd.read_csv('https://raw.githubusercontent.com/Nhrkr/Predicting-Social-Unrest/master/acled_all.csv')
data=data[~data.data_id.duplicated(keep='first')]

In [0]:
data.head()

,data_id,event_type,sub_event_type,actor1,interaction,admin1,fatalities,timestamp
0,4576055,Riots,Mob violence,Rioters (India),15,Kerala,0,1552576416
2,4576056,Riots,Mob violence,Rioters (India),57,Assam,0,1552576416
4,4576057,Strategic developments,Disrupted weapons use,IUML: Indian Union Muslim League,30,Kerala,0,1552576416
5,4576058,Violence against civilians,Attack,Unidentified Armed Group (India),37,Bihar,1,1552576416
7,4576059,Violence against civilians,Attack,Mambapur Communal Militia (India),47,Telangana,1,1552576416


In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
enc = preprocessing.OneHotEncoder()
from sklearn.compose import ColumnTransformer

In [0]:
ct = ColumnTransformer(
    [('data', preprocessing.OneHotEncoder(sparse=False), ['event_type','sub_event_type','actor1','interaction', 'admin1']),],  # the column numbers I want to apply this to
#     remainder='passthrough'  # This leaves the rest of my columns in place
)
features_onehot= ct.fit_transform(data)
features_onehot= pd.DataFrame(features_onehot, columns= ct.get_feature_names())
print(type(features_onehot))

<class 'pandas.core.frame.DataFrame'>


In [0]:
features_onehot=features_onehot.astype(float)

In [0]:
timestamps= pd.DataFrame(pd.to_datetime(data['timestamp'], unit='s'))

In [0]:
timestamps.head()

,timestamp
0,2019-03-14 15:13:36
2,2019-03-14 15:13:36
4,2019-03-14 15:13:36
5,2019-03-14 15:13:36
7,2019-03-14 15:13:36


In [0]:
final_df= timestamps.join(features_onehot)

In [0]:
final_df.head()

,timestamp,data__x0_Battles,data__x0_Explosions/Remote violence,data__x0_Protests,data__x0_Riots,data__x0_Strategic development,data__x0_Strategic developments,data__x0_Violence against civilians,data__x1_Abduction/forced disappearance,data__x1_Agreement,...,data__x4_Puducherry,data__x4_Punjab,data__x4_Rajasthan,data__x4_Sikkim,data__x4_Tamil Nadu,data__x4_Telangana,data__x4_Tripura,data__x4_Uttar Pradesh,data__x4_Uttarakhand,data__x4_West Bengal
0,2019-03-14 15:13:36,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-03-14 15:13:36,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-03-14 15:13:36,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,2019-03-14 15:13:36,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2019-03-14 15:13:36,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
x, y= create_data_set(final_df)

TypeError: ignored

In [0]:
from statsmodels.tsa.vector_ar.var_model import VAR

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [0]:
train = np.asarray(features_onehot[:int(0.8*(len(features_onehot)))])
valid = np.asarray(features_onehot[int(0.8*(len(features_onehot))):])


In [0]:
model = VAR(endog=train)
model_fit = model.fit()

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/vector_ar/var_model.py:461: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  params = np.linalg.lstsq(z, y_sample)[0]


In [0]:
prediction = model_fit.forecast(model_fit.y, steps=10)

In [0]:
len(prediction)

10

In [0]:
pred = pd.DataFrame(index=range(0,len(prediction)),columns=list(features_onehot.columns.values))
for j in range(0,13):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]


In [0]:
pred.head()

,data__x0_Battles,data__x0_Explosions/Remote violence,data__x0_Protests,data__x0_Riots,data__x0_Strategic development,data__x0_Strategic developments,data__x0_Violence against civilians,data__x1_Abduction/forced disappearance,data__x1_Agreement,data__x1_Armed clash,...,data__x4_Puducherry,data__x4_Punjab,data__x4_Rajasthan,data__x4_Sikkim,data__x4_Tamil Nadu,data__x4_Telangana,data__x4_Tripura,data__x4_Uttar Pradesh,data__x4_Uttarakhand,data__x4_West Bengal
0,0.0197449,0.00613785,0.854431,0.0997314,0.000365615,0.00634766,0.0132294,0.000101089,5.56633e-05,0.0194092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.16384e+09,8.62962e+08,-1.85174e+09,9.15553e+09,-1.16616e+08,-7.87057e+09,-2.3434e+09,7.37343e+07,7.56163e+06,2.16525e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.89293e+08,3.37274e+08,2.42995e+08,1.8243e+10,1.33947e+08,-1.47826e+10,-4.75349e+09,1.02223e+08,8.20673e+06,5.82576e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.02783e+19,-4.76533e+18,2.71196e+18,-1.35605e+19,3.31656e+17,-2.05564e+19,4.6117e+19,-8.12103e+17,8.08989e+16,-1.02797e+19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-5.32624e+19,-3.95568e+19,3.29866e+19,-1.8694e+20,1.24292e+18,5.72092e+19,1.88384e+20,-4.66424e+18,-1.44784e+17,-5.33187e+19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
valid

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [0]:
for i in list(features_onehot.columns.values):
    print('rmse value for', i, 'is : ', math.sqrt(mean_squared_error(pred[i], valid[i])))

IndexError: ignored

In [0]:
# Nice name